# 畳み込みオートエンコーダー

## GoogleDriveマウント

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## 必要なモジュールのインポート

In [ ]:
import torch
import torchvision
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.datasets import CIFAR10
import numpy as np
import matplotlib.pyplot as plt

## データパス設定

In [ ]:
CIFAR10_path = "/content/drive/My Drive/Colab Notebooks/autoencorder_app/data/CIFAR10/"

##データローダー定義

In [ ]:
#CIFAR10内のautomobile画像のみ取得するためのsampler
def get_auto_sampler(trainset):
    target = [1] #1はautomobile。
    mask = [s[1] in target for s in trainset] #automobileのみtrueにマスクする=maskは、automobileのみ1の重みベクトル

    print('合計数：', sum(mask))

    #重み付きsampler：trueの画像のみ対象にする。
    sampler = torch.utils.data.sampler.WeightedRandomSampler(mask, len(trainset))

    return sampler

transform = transforms.Compose(
    [
        transforms.Resize(256),  # (256, 256) で切り抜く。
        transforms.CenterCrop(224),  # 画像の中心に合わせて、(224, 224) で切り抜く
        transforms.ToTensor(),  # テンソルにする。
        transforms.Normalize( # 標準化する。
            mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]
        ),  
    ]
)
trainset = CIFAR10(root=CIFAR10_path, train=True, transform=transform, download=True)
testset = CIFAR10(root=CIFAR10_path, train=False, transform=transform, download=True)

#重み付きsampler
auto_sampler = get_auto_sampler(trainset)

# batch_size = 50
batch_size = 100
# trainloader = DataLoader(trainset, batch_size=batch_size, shuffle=True)
trainloader = DataLoader(trainset, batch_size=batch_size, sampler=auto_sampler) #samplerはshuffleと一緒に使用できない。

print('学習用データセット：',trainset.data.shape)
print('クラス：',trainset.classes)

#下記作成したtrainloaderからデータ抽出して画像確認。
# #指定したバッチサイズ分画像を取得
# train_iter = iter(trainloader)
# images, labels = train_iter.next()

# def imshow(img):
#     img = img / 2 + 0.5     # unnormalize
#     npimg = img.numpy()
#     plt.imshow(np.transpose(npimg, (1, 2, 0)))

# #trainloaderの画像を表示して、車のみであること確認。
# imshow(torchvision.utils.make_grid(images)) 

## 学習用関数定義

In [ ]:
def train(model, criterion, optimizer, epochs, trainloader):
    losses = []
    output_and_label = []

    for epoch in range(1, epochs+1):
        print(f'epoch: {epoch}, ', end='')
        running_loss = 0.0
        for counter, (img, _) in enumerate(trainloader, 1):

            optimizer.zero_grad()

            output = model(img)

            loss = criterion(output, img)

            loss.backward()

            optimizer.step()

            running_loss += loss.item()
        avg_loss = running_loss / counter

        losses.append(avg_loss)

        print('loss:', avg_loss)
        output_and_label.append((output, img))

    print('finished')

    return output_and_label, losses

## オートエンコーダーを実装するクラス定義

In [ ]:
class AE(torch.nn.Module):
    def __init__(self, enc, dec):
        super().__init__()
        self.enc = enc
        self.dec = dec
    def forward(self, x):
        x = self.enc(x)
        x = self.dec(x)
        return x

## エンコーダーとディコーダー定義

In [ ]:
#エンコーダー設定
enc = torch.nn.Sequential(
    #第一引数：入力チェンネル数、第二引数：出力チャンネル数＝画像の走査に使用するカーネルの数。
    torch.nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, stride=1, padding=1),
    torch.nn.ReLU(),
    torch.nn.MaxPool2d(2),
    torch.nn.Conv2d(in_channels=16, out_channels=8, kernel_size=3, stride=1, padding=1),
    torch.nn.ReLU(),
    torch.nn.MaxPool2d(2) #カーネルサイズに2×2を指定。
)

#ディコーダー設定
dec = torch.nn.Sequential(
    torch.nn.ConvTranspose2d(in_channels=8, out_channels=16, kernel_size=2, stride=2),
    torch.nn.ReLU(),
    torch.nn.ConvTranspose2d(in_channels=16, out_channels=3, kernel_size=2, stride=2),
    torch.nn.Sigmoid()
)

## モデルの学習学習

In [ ]:
model = AE(enc, dec)
criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

EPOCHS = 100
# EPOCHS = 50
#input_size = 3 * 32 * 32
#バッチサイズ50、エポック50→0.20始まりで0.18ぐらいで終わる = 前処理で(256, 256) で切り抜き無し。
#バッチサイズ100、エポック100→0.51始まりで0.18ぐらいで終わる = 前処理で(256, 256) で切り抜き無し。
#バッチサイズ100、エポック100→前処理で(256, 256) で切り抜き有り実施してみる。

output_and_label, losses = train(model, criterion, optimizer, EPOCHS, trainloader)
print("学習終了")

# GPUで学習したモデルの保存
save_gpu_path = "/content/drive/My Drive/Colab Notebooks/autoencorder_app/trained_model/autoencorder_gpu.pth"
torch.save(model.state_dict(), save_gpu_path)

# CPUに変更したモデルを保存
save_cpu_path = "/content/drive/My Drive/Colab Notebooks/autoencorder_app/trained_model/autoencorder_cpu.pth"
torch.save(model.to('cpu').state_dict(), save_cpu_path)       
print("保存終了")

In [ ]:
# GPUで学習したモデルの保存
save_gpu_path = "/content/drive/My Drive/Colab Notebooks/autoencorder_app/trained_model/autoencorder_gpu.pth"
torch.save(model.state_dict(), save_gpu_path)

# 念の為念の為CPUに変更したものも保存
save_cpu_path = "/content/drive/My Drive/Colab Notebooks/autoencorder_app/trained_model/autoencorder_cpu.pth"
torch.save(model.to('cpu').state_dict(), save_cpu_path)       
print("保存終了")

## Lossの確認

In [ ]:
plt.plot(losses)

## inputととoutputの画像比較

In [ ]:
img, org = output_and_label2[-1]
imshow(org)
imshow(img)